In [12]:
print("giskard_testing")

giskard_testing


In [15]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
loader=PyPDFLoader("attention_is_all_you_need.pdf")
attention=loader.load()

In [18]:
documents=text_splitter.split_documents(attention)
vectorstore=DocArrayInMemorySearch.from_documents(
    documents,embedding=OpenAIEmbeddings()
)

In [19]:
vectorstore

In [20]:
import pandas as pd 
from giskard.rag import KnowledgeBase 

df=pd.DataFrame([file.page_content for file in documents],columns=["text"])
knowledge_base=KnowledgeBase(df)

In [21]:
df.head(10)

,text
0,"Provided proper attribution is provided, Googl..."
1,mechanism. We propose a new simple network arc...
2,best models from the literature. We show that ...
3,efficient inference and visualizations. Lukasz...
4,"1 Introduction\nRecurrent neural networks, lon..."
5,sequential nature precludes parallelization wi...
6,"In this work we propose the Transformer, a mod..."
7,"in the distance between positions, linearly fo..."
8,textual entailment and learning task-independe...
9,"Here, the encoder maps an input sequence of sy..."


In [22]:
from giskard.rag import generate_testset 

testset=generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description="A Chat bot answering questions related to Pdf uploaded by Users"
)
test_set_df=testset.to_pandas()

2024-12-25 22:06:14,597 pid:7268 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


c:\Users\Hp\Desktop\Rag_Langchain\virtualenv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


2024-12-25 22:06:20,419 pid:7268 MainThread giskard.rag  INFO     Found 3 topics in the knowledge base.


Generating questions: 100%|██████████| 60/60 [02:41<00:00,  2.70s/it]


In [23]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")

Question 1: What is the complexity per layer for self-attention and recurrent layers?
Reference answer: The complexity per layer for self-attention is O(n2 · d), and for recurrent layers, it is O(n · d2).
Reference context:
Document 20: Table 1: Maximum path lengths, per-layer complexity and minimum number of sequential operations
for different layer types. n is the sequence length, d is the representation dimension, k is the kernel
size of convolutions and r the size of the neighborhood in restricted self-attention.
Layer Type Complexity per Layer Sequential Maximum Path Length
Operations
Self-Attention O(n2 · d) O(1) O(1)
Recurrent O(n · d2) O(n) O(n)
Convolutional O(k · n · d2) O(1) O(logk(n))
Self-Attention (restricted) O(r · n · d) O(1) O(n/r)
3.5 Positional Encoding
Since our model contains no recurrence and no convolution, in order for the model to make use of the
order of the sequence, we must inject some information about the relative or absolute position of the
tokens in the 

In [24]:
testset.save("test-set.json")

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based on provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>
Question:{input}
"""
)

query="Who are authors of Attention is all you need paper ?"
retrieved_results=vectorstore.similarity_search(query)
context=retrieved_results[0].page_content
context

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurren

In [26]:
chain=prompt|ChatOpenAI()|StrOutputParser()
response=chain.invoke(
    {
        "context":context,
        "input":query
    }
)
response

'The authors of the "Attention Is All You Need" paper are Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin.'

In [27]:
def answer_fn(query,history=None):
    query=query
    retrieved_results=vectorstore.similarity_search(query)
    context=retrieved_results[0].page_content
    chain=prompt|ChatOpenAI()|StrOutputParser()
    return chain.invoke({"context":context,"input":query })

In [28]:
from giskard.rag import evaluate

report=evaluate(answer_fn,testset=testset,knowledge_base=knowledge_base)

CorrectnessMetric evaluation: 100%|██████████| 60/60 [01:09<00:00,  1.16s/it]


In [29]:
display(report)

Loading BokehJS ...

In [30]:
report.correctness_by_question_type()

,correctness
question_type,
complex,0.5
conversational,0.1
distracting element,0.7
double,0.9
simple,0.7
situational,0.7
